# CHAPTER 2 - The Tools of the Trade in Quantum Computing - PennyLane Code

*Note*: You may skip the following three cells if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.



In [ ]:
conda create --name pennylane_env --file requirements_pennylane.txt

In [ ]:
pip install -r requirements_pennylane.txt

In [1]:
import pennylane as qml

In [2]:
dev = qml.device("default.qubit", wires=2)  # Add shots for realism

In [3]:
def qc():
    qml.PauliX(wires = 0)
    qml.Hadamard(wires = 0)
    return qml.state()

In [4]:
# ✅ Updated QNode binding (New API)
# Assemble the circuit & the device.
qcirc = qml.qnode(dev)(qc)  # Bind the circuit to the device properly

# ✅ Updated execution (explicit call for clarity)
result = qcirc()
print("Circuit Output:", result)  # ✅ Print for verification

Circuit Output: [ 0.70710678+0.j  0.        +0.j -0.70710678+0.j  0.        +0.j]


In [5]:
@qml.qnode(dev)  # We add this decorator to use the device dev.
def qcirc():
    qml.PauliX(wires=0)
    qml.Hadamard(wires=0)
    return qml.state()

# Now qcirc is already a QNode. We can just run it!
result = qcirc()
print("Quantum State:", result)  # ✅ Print for verification

Quantum State: [ 0.70710678+0.j  0.        +0.j -0.70710678+0.j  0.        +0.j]


In [6]:
dev = qml.device('default.qubit', wires = 1)
@qml.qnode(dev)
def qcirc(theta):
    qml.RX(theta, wires = 0)
    return qml.state()

In [7]:
print(qml.draw(qcirc)(theta = 2))

0: ──RX(2.00)─┤  State


In [8]:
dev = qml.device('default.qubit', wires = 3)

# Get probabilities
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 1)
    return qml.probs(wires = [1, 2]) # Only the last 2 wires.
prob = qcirc()
print("Probs. wires [1, 2] with H in wire 1:", prob)

# Get a sample, not having specified shots in the device.
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 0)
    return qml.sample(wires = 0) # Only the first wire.
s1 = qcirc(shots = 4) # We specify the shots here.
print("Sample 1 after H:", s1)

# Get a sample with shots in the device.
dev = qml.device('default.qubit', wires = 2, shots = 4)
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires=0)
    return qml.sample() # Will sample all wires.
s2 = qcirc()
print("Sample 2 after H x I:", s2)

Probs. wires [1, 2] with H in wire 1: [0.5 0.  0.5 0. ]
Sample 1 after H: [1 1 0 0]
Sample 2 after H x I: [[1 0]
 [0 0]
 [0 0]
 [1 0]]


In [9]:
dev = qml.device('qiskit.aer', wires = 2)
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 0)
    return qml.probs(wires = 0)
s = qcirc()
print("The probabilities are", s)

The probabilities are [0.48144531 0.51855469]


*Note*: In the following cell, you need to replace "1234" with your actual IBM token. Refer to *Appendix D* in the book for instructions on how to create an account and get your token. Be very careful not to disclose your token to anyone!

In [10]:
YOUR_IBM_QUANTUM_API_TOKEN = "dabbdf2b97f2d9f3b5a903238970a19cebdfcb8aec0804f3f7ac50f53e2db340b40d158da5808e139d6e132796dc4e61ea7e9479826ed0bd3ad5ad6b60a01cac"

In [15]:
import pennylane as qml
from qiskit_ibm_runtime import QiskitRuntimeService

# ✅ Save and load IBM Quantum account credentials (only needed once)
QiskitRuntimeService.save_account(channel="ibm_quantum", token=YOUR_IBM_QUANTUM_API_TOKEN, overwrite=True)

# ✅ Load the saved credentials
service = QiskitRuntimeService(channel="ibm_quantum", instance='ibm-q/open/main')

# ✅ Select a real IBM quantum backend (not a simulator)
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=2)

print(f"Selected Backend: {backend.name}")  # ✅ Debugging: Ensure correct backend selection

# ✅ Initialize the PennyLane device
dev = qml.device("qiskit.remote", wires=2, backend=backend)  

# ✅ Define a simple quantum circuit
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires=0)
    return qml.probs(wires=0)

# ✅ Run the quantum circuit and print the results
result = qcirc()
print("Quantum Circuit Output:", result)


Selected Backend: ibm_brisbane
Quantum Circuit Output: [0.48046875 0.51953125]


In [16]:
pip freeze > requirements_pennylane.txt

Note: you may need to restart the kernel to use updated packages.


In [14]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Load your IBM Quantum account
service = QiskitRuntimeService(channel="ibm_quantum")

# Print available instances
print(service.instances())

['ibm-q/open/main']
